# Black-Scholes Baseline

We compute implied vols across strikes from real SPX options data.
The goal is simple: show that BS implies a flat vol surface. Markets don't.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import sys
sys.path.append('../')
from models.black_scholes import bs_price, implied_vol

In [ ]:
# Pull real SPX options from Yahoo Finance
spx = yf.Ticker('^SPX')
S = spx.history(period='1d')['Close'].iloc[-1]

# Pick the first available expiry
expiry = spx.options[2]
chain = spx.option_chain(expiry)
calls = chain.calls

print(f'Spot: {S:.2f} | Expiry: {expiry}')
calls[['strike', 'lastPrice', 'impliedVolatility']].head(10)

In [ ]:
# Keep only liquid strikes around ATM
calls = calls[(calls['strike'] > S * 0.85) & (calls['strike'] < S * 1.15)]
calls = calls[calls['lastPrice'] > 0.5].reset_index(drop=True)

# Time to maturity in years
from datetime import datetime
T = (datetime.strptime(expiry, '%Y-%m-%d') - datetime.today()).days / 365
r = 0.05

strikes = calls['strike'].values
prices  = calls['lastPrice'].values

In [ ]:
# Compute implied vols by inverting BS numerically
ivols = np.array([implied_vol(p, S, K, T, r) for p, K in zip(prices, strikes)])

# Remove NaNs
mask = ~np.isnan(ivols)
strikes, ivols = strikes[mask], ivols[mask]

In [ ]:
# The smile
atm_vol = ivols[np.argmin(np.abs(strikes - S))]

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(strikes, ivols * 100, 'o-', color='steelblue', linewidth=2, label='Implied vol (market)')
ax.axhline(y=atm_vol * 100, color='tomato', linestyle='--', linewidth=1.5, label='BS flat vol (ATM)')
ax.axvline(x=S, color='gray', linestyle=':', alpha=0.6, label='ATM spot')

ax.set_xlabel('Strike')
ax.set_ylabel('Implied Volatility (%)')
ax.set_title(f'SPX Volatility Smile  |  Expiry {expiry}  |  Spot {S:.0f}')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/vol_smile.png', dpi=150)
plt.show()

The red line is what BS predicts: flat vol everywhere.
The blue curve is what the market actually prices.
This gap is exactly what Heston and Rough Bergomi are built to close.